In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Mar  21 19:42:02 2020

@author: yihao.he
"""

from bs4 import BeautifulSoup  
import pandas as pd
from tqdm import tqdm
import math
import requests  
import lxml
import re
import time


# 房源面积-总价散点图
# 各行政区均价
# 均价最贵的10个地段
# 均价最贵的10个小区
# 户型分布
# 标题文本词云图


from pyecharts.charts import *
from pyecharts import options as opts
from pyecharts.commons.utils import JsCode
from jieba import posseg as psg                         #中文分词解释
import collections


# 读入CSV 命名为data
data = pd.read_csv('.\\JN_DataWarehouse\\foshan_lianjiafangyuan\\lianjia-foshanfangyuan.csv')
print('start:',data.head())


# 房源面积-总价散点图
# 💥最贵的一套房源是位于宝安-曦城的房源，总价8800W；

scatter = (Scatter(init_opts=opts.InitOpts(theme='dark'))
           .add_xaxis(data['hourseSize'])
           .add_yaxis("房价", data['total_price'])
           .set_series_opts(label_opts=opts.LabelOpts(is_show=False),
                           markpoint_opts=opts.MarkPointOpts(data=[opts.MarkPointItem(type_="max", name="最大值"),]))
           .set_global_opts(
               legend_opts=opts.LegendOpts(is_show=False),
               title_opts=opts.TitleOpts(title="佛山二手房 总价-面积 散点图"),
               xaxis_opts=opts.AxisOpts(
                   name='面积',
                   # 设置坐标轴为数值类型
                   type_="value", 
                   # 不显示分割线
                   splitline_opts=opts.SplitLineOpts(is_show=False)),
               yaxis_opts=opts.AxisOpts(
                   name='总价',
                   name_location='middle',
                   # 设置坐标轴为数值类型
                   type_="value",
                   # 默认为False表示起始为0
                   is_scale=True,
                   splitline_opts=opts.SplitLineOpts(is_show=False),),
               visualmap_opts=opts.VisualMapOpts(is_show=True, type_='color', min_=100, max_=1000)
    ))

scatter.render_notebook()                      #直接在jupyter notebook 显示
#scatter.render('散点图.html')                #生成html文件



start:    Unnamed: 0 area                      title  community position    tax  \
0           0  禅城区  急售！保利香槟花园 满五唯一 精装修刚需轻奢大三房     保利香槟花园       张槎    NaN   
1           1  禅城区            满5唯一税费少 小区环境好又靓       翠影华庭       张槎  房本满五年   
2           2  禅城区     东海银湾精装3房 满五年税少 总价低 快上车     长信东海银湾       朝安    NaN   
3           3  禅城区        智博丽海花园 高楼层东南向 带装修出售  联诚·智博丽海花园       石湾    NaN   
4           4  禅城区       东南向精致3房， 安静望花园  随时看房       时代年华       澜石    NaN   

   total_price  unit_price hourseType  hourseSize direction fitment  
0        148.0     16323.0       3室2厅       90.67         北      精装  
1         70.0     10494.0       2室1厅       66.71         南      简装  
2        170.0     20988.0       3室1厅       81.00         北      精装  
3        152.0     16766.0       3室2厅       90.66        东南      精装  
4        160.0     19278.0       3室1厅       83.00         南      精装  


In [2]:

# 各行政区均价
# 最贵的是南山区，整体均价9.2W/平米；
# 最便宜的是坪山区，均价3.5W/平米；

temp = data.groupby(['area'])['unit_price'].mean().reset_index()
data_pair = [(row['area'], round(row['unit_price']/10000, 1)) for _, row in temp.iterrows()]

map_ = (Map(init_opts=opts.InitOpts(theme='dark'))
        .add("二手房均价", data_pair, '佛山', is_roam=False)
        .set_series_opts(label_opts=opts.LabelOpts(is_show=True))
        .set_global_opts(
            title_opts=opts.TitleOpts(title="佛山各行政区二手房均价"),
            legend_opts=opts.LegendOpts(is_show=False),
            tooltip_opts=opts.TooltipOpts(formatter='{b}:{c}万元'),
            visualmap_opts=opts.VisualMapOpts(min_=0, max_=3)
        )
       )

        
map_.render_notebook()
#map_.render('地理图.html')



In [3]:
# 均价最高的10个小区
# 该小区内在售房源至少3套才统计

temp = data.groupby(['community'])['unit_price'].agg(['mean', 'count']).reset_index()

# 该小区内至少3套在售房源才统计
data_pair = sorted([(row['community'], round(row['mean']/10000, 1)) if row['count']>=3 else (0, 0)
                    for _, row in temp.iterrows()], key=lambda x: x[1], reverse=True)[:10]

bar = (Bar(init_opts=opts.InitOpts(theme='dark'))
       .add_xaxis([x[0] for x in data_pair[::-1]])
       .add_yaxis('二手房均价', [x[1] for x in data_pair[::-1]])
       .set_series_opts(label_opts=opts.LabelOpts(is_show=True, 
                                                       position='insideRight',
                                                       font_style='italic'),
                            itemstyle_opts=opts.ItemStyleOpts(
                                color=JsCode("""new echarts.graphic.LinearGradient(1, 0, 0, 0, 
                                             [{
                                                 offset: 0,
                                                 color: 'rgb(0,206,209)'
                                             }, {
                                                 offset: 1,
                                                 color: 'rgb(218,165,32)'
                                             }])"""))
                            )
       .set_global_opts(
           title_opts=opts.TitleOpts(title="佛山二手房均价TOP 10小区"),
           legend_opts=opts.LegendOpts(is_show=False),
           tooltip_opts=opts.TooltipOpts(formatter='{b}:{c}万元'),
           xaxis_opts=opts.AxisOpts(max_=5),
       )
       .reversal_axis()
      )

bar.render_notebook()
#bar.render('柱状图10小区.html')

In [4]:



# 均价最高的10个地段

temp = data.groupby(['position'])['unit_price'].mean().reset_index()
data_pair = sorted([(row['position'], round(row['unit_price']/10000, 1))
                    for _, row in temp.iterrows()], key=lambda x: x[1], reverse=True)[:10]

bar = (Bar(init_opts=opts.InitOpts(theme='dark'))
       .add_xaxis([x[0] for x in data_pair])
       .add_yaxis('二手房均价', [x[1] for x in data_pair])
       .set_series_opts(label_opts=opts.LabelOpts(is_show=True, font_style='italic'),
                            itemstyle_opts=opts.ItemStyleOpts(
                                color=JsCode("""new echarts.graphic.LinearGradient(0, 1, 0, 0, 
                                             [{
                                                 offset: 0,
                                                 color: 'rgb(0,206,209)'
                                             }, {
                                                 offset: 1,
                                                 color: 'rgb(218,165,32)'
                                             }])"""))
                            )
       .set_global_opts(
           title_opts=opts.TitleOpts(title="佛山二手房均价TOP 10地段"),
           legend_opts=opts.LegendOpts(is_show=False),
           tooltip_opts=opts.TooltipOpts(formatter='{b}:{c}万元'))
      )

bar.render_notebook()
#bar.render('柱状图10地段.html')

In [5]:

# 户型分布
# 三室依然是主力；
# 在佛山这种寸土寸金的地方，小户型占比也很多；

temp = data.groupby(['hourseType'])['area'].count().reset_index()
data_pair = sorted([(row['hourseType'], row['area'])
                    for _, row in temp.iterrows()], key=lambda x: x[1], reverse=True)[:10]

pie = (Pie(init_opts=opts.InitOpts(theme='dark'))
       .add('', data_pair,
            radius=["30%", "75%"],
            rosetype="radius")
       .set_global_opts(title_opts=opts.TitleOpts(title="佛山二手房 户型分布"),
                       legend_opts=opts.LegendOpts(is_show=False),)
       .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {d}%"))
      )

pie.render_notebook()
#pie.render('户型占比图.html')



In [6]:
# 词云图
# 来看看房源标题中出现最多的词语

word_list = []
stop_words = ['花园','业主','出售']
string =  str(''.join([i for i in data['title'] if isinstance(i, str)]))


words = psg.cut(string)
for x in words:
    if len(x.word)==1:
        pass
    elif x.flag in ('m', 'x'):
        pass
    elif x.word in stop_words:
        pass
    else:
        word_list.append(x.word)


data_pair = collections.Counter(word_list).most_common(100)


wc = (WordCloud()
      .add("", data_pair, word_size_range=[20, 100], shape='triangle')
      .set_global_opts(title_opts=opts.TitleOpts(title="房源描述词云图"))
    )

wc.render_notebook()
#wc.render('词云图.html')

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\pc-01\AppData\Local\Temp\jieba.cache
Loading model cost 1.819 seconds.
Prefix dict has been built successfully.
